In [59]:
import glob
import pandas as pd
import numpy as np
import re

# Project idea and description...


Tadatadatada

# Data aquisition

In the first ... 

We need to get all the following keywords from twitter :

abu sayyaf,
afghanistan,
agro,
al-qaeda,
al-qaeda in the arabian peninsula,
al-qaeda in the islamic maghreb,
al-shabaab,
ammonium nitrate,
attack,
biological weapon,
car bomb,
chemical weapon,
conventional weapon,
dirty bomb,
eco-terrorism,
environmental terrorism,
euskadi ta askatasuna,
extremism,
farc,
fundamentalism,
hamas,
hezbollah,
improvised explosive device,
iran,
iraq,
irish republican army,
islamist,
jihad,
nationalism,
nigeria,
nuclear,
nuclear enrichment,
pakistan,
palestine liberation front,
pirates,
plo,
political radicalism,
recruitment,
somalia,
suicide attack,
suicide bomber,
taliban,
tamil tigers,
tehrik-i-taliban pakistan,
terror,
terrorism,
weapons-grade,
yemen,

## Analysis of the tweets

### Interrupted time series with regression
We will now focus on getting an understanding of the tweet distribution over time, and how the massive revelations of online surveillance in June 2013 might have caused a chilling effect. We will follow the paper's original way of doing the interrupted time series, with regression analysis.

First, let's see what those big files yield by reading one. We need to first find all the archives.

In [42]:
archive_pathnames = glob.glob('./data/*.gz')
print(f"Found {len(archive_pathnames)} archives")

Found 44 archives


Now, what is in the first archive ?

In [46]:
df = pd.read_csv(archive_pathnames[0])
print(df.shape)
df.head()

(9299, 22)


,id,conversation_id,created_at,date,time,user_id,username,name,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,link,retweet,video,thumbnail,reply_to
0,506278868614148097,506278868614148097,2014-09-01 05:14:23 CEST,2014-09-01,05:14:23,8.343178e+07,starman1981,truckerstrong news,ISIS laptop reveals terror group were working ...,en,...,[],0,0.0,0.0,[],https://twitter.com/starman1981/status/5062788...,False,False,NaN,[]
1,506272708037214208,506272708037214208,2014-09-01 04:49:54 CEST,2014-09-01,04:49:54,7.134575e+08,patriotsorg,Patriots Billboard,#SEIZED ISIS LAPTOP REVEALS THEY’RE DEVELOPING...,en,...,[],0,3.0,1.0,['seized'],https://twitter.com/PatriotsOrg/status/5062727...,False,False,NaN,[]
2,506263382459428864,506263382459428864,2014-09-01 04:12:50 CEST,2014-09-01,04:12:50,2.630728e+08,mlgeek,My Little Geek,@bjorntipling Perhaps Swift is a biological we...,en,...,[],0,0.0,0.0,[],https://twitter.com/mlgeek/status/506263382459...,False,False,NaN,[]
3,506261754197061632,506261754197061632,2014-09-01 04:06:22 CEST,2014-09-01,04:06:22,2.510451e+09,msgubot,Cédric Moro (backup),G+ #ebola USA created Ebola virus as biologica...,en,...,[],0,0.0,0.0,['ebola'],https://twitter.com/MSGUBOT/status/50626175419...,False,False,NaN,[]
4,506239829819281408,506239829819281408,2014-09-01 02:39:15 CEST,2014-09-01,02:39:15,7.134575e+08,patriotsorg,Patriots Billboard,#SEIZED ISIS LAPTOP REVEALS THEY’RE DEVELOPING...,en,...,[],1,2.0,1.0,['seized'],https://twitter.com/PatriotsOrg/status/5062398...,False,False,NaN,[]


We see a lot of information there. But what we are interested in is the user interactions around a topic that they might the government to track, for example. Therefore, we can count the number of tweets themselves, not any of their content or information, but also the number of likes and rerplies ! Each of these actions can make the user fear such surveillance. Retweets also, but because we also collect the retweets themselves, they are already there ! Therefore, we will first count the number of tweets per month, with the number of likes and retweets added as well.

In [58]:
# Keep only interesting columns and sum them all together by month. We set to parse dates so we can group by month
df = pd.read_csv(archive_pathnames[0], usecols=["date", "likes_count", "replies_count"], parse_dates=["date"], lineterminator='\n')
df["tweet_count"] = 1
grouped_df = df.set_index('date').groupby(pd.Grouper(freq='M')).sum()
grouped_df["user_interactions"] = grouped_df["likes_count"] +  grouped_df["tweet_count"] +  grouped_df["replies_count"]

grouped_df.head()

,replies_count,likes_count,tweet_count,user_interactions
date,,,,
2012-01-31,49,51,207,307
2012-02-29,33,27,177,237
2012-03-31,37,10,192,239
2012-04-30,40,32,156,228
2012-05-31,46,17,237,300


We will want to merge all twitter keyworkds into the same dataframe, therefore we will give the name of the keywork to the column where all values are summed, instead of just user_interactions. Let's extract the name from the archive path.

In [60]:
print(archive_pathnames[0])

./data/biological weapon_full.gz


In [61]:
# Knowing what the file path looks like, we can extract the name
name = re.search(r"(?<=data/).*?(?=_full)", archive_pathnames[0]).group(0)
name

'biological weapon'

In [64]:
# Just set this name to the column and go to the next zip !
grouped_df.rename(columns={"user_interactions": name}, inplace=True)
grouped_df.head()

,replies_count,likes_count,tweet_count,biological weapon
date,,,,
2012-01-31,49,51,207,307
2012-02-29,33,27,177,237
2012-03-31,37,10,192,239
2012-04-30,40,32,156,228
2012-05-31,46,17,237,300


Let's now implement a for loop to aggregate all the data.

In [ ]:
# print(pd.read_csv(archive_pathnames[0], usecols=["date", "likes_count"]))
monthly_counts = pd.DataFrame([])

for archive_pathname in archive_pathnames:
    print(f"Reading {archive_pathname} file")
    df = pd.read_csv(archive_pathname, usecols=["date", "likes_count", "replies_count"], parse_dates=["date"], lineterminator='\n')
    print(f"Shape is : {df.shape} \n")

    df["tweet_count"] = 1
    df = df.set_index('date').groupby(pd.Grouper(freq='M')).sum()
    df["user_interactions"] = df["likes_count"] +  df["tweet_count"] +  df["replies_count"]
    name = re.search(r"(?<=data/).*?(?=_full)", archive_pathname).group(0)

    df.rename(columns={"user_interactions": name}, inplace=True)
    
    monthly_counts = pd.concat([monthly_counts, df[name]], axis=1)

In [76]:
print(f"We have {np.round(monthly_counts.sum().sum()/1000000, 2)} million actions !")

We have 130.87 million actions !
